In [76]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from PIL import Image
import pandas as pd

In [77]:
picsdir = 'static'
df = pd.read_csv("markup.csv")
datalen = len(df)

In [78]:
validation_size = 0.25
autotune = tf.data.experimental.AUTOTUNE
img_size = 299
batch_size = 8
dropout = 0.2
lr = 1e-3
decay_steps = 100
decay_rate = 0.96
epochs = 35

In [79]:
def get_image(filename):
    image = tf.image.decode_jpeg(tf.io.read_file(f"{picsdir}/{filename}"), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, img_size, img_size), dtype = tf.float32)
    return tf.keras.applications.inception_v3.preprocess_input(image)

In [80]:
x = [get_image(filename) for filename in df["Filename"].values]

In [81]:
y = [[int(index == mark) for index in range(10)] for mark in df["Mark"].values]

In [82]:
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).prefetch(buffer_size=autotune)

In [83]:
train_part = 0.75
split_index = int(datalen * train_part // batch_size)
trainset = dataset.take(split_index)
validset = dataset.skip(split_index)

In [84]:
model = tf.keras.models.load_model('models/InceptionV3.h5')
for layer in model.layers[300:]:
    layer.trainable = True

In [85]:
model= tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(img_size, img_size, 3)),
        tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
        model,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout, name='top_dropout'),
        tf.keras.layers.Dense(32, activation='elu'),
        tf.keras.layers.Dense(10,'softmax')
    ]
)

In [86]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=decay_steps, decay_rate=decay_rate,staircase=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', patience=5, restore_best_weights=True)

class Callbackss(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if logs.get('loss') < 0.063:
            self.model.stop_training = True
            
call = Callbackss()

In [87]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [88]:
history=model.fit(trainset,validation_data=validset, epochs=epochs, verbose=1, callbacks=[early_stop, call], use_multiprocessing=True, workers=-1)

Epoch 1/35
67/67 [==============================] - 219s 3s/step - loss: 0.4835 - accuracy: 0.1884 - val_loss: 11.5499 - val_accuracy: 0.1383
Epoch 2/35
67/67 [==============================] - 212s 3s/step - loss: 0.3664 - accuracy: 0.1716 - val_loss: 2.6401 - val_accuracy: 0.2021
Epoch 3/35
67/67 [==============================] - 174s 3s/step - loss: 0.3568 - accuracy: 0.2183 - val_loss: 0.3144 - val_accuracy: 0.2340
Epoch 4/35
67/67 [==============================] - 85s 1s/step - loss: 0.3395 - accuracy: 0.2090 - val_loss: 985350.7500 - val_accuracy: 0.0160
Epoch 5/35
67/67 [==============================] - 85s 1s/step - loss: 0.3387 - accuracy: 0.2239 - val_loss: 5.1562 - val_accuracy: 0.1968
Epoch 6/35
67/67 [==============================] - 88s 1s/step - loss: 0.3380 - accuracy: 0.1996 - val_loss: 56.7846 - val_accuracy: 0.0904
Epoch 7/35
67/67 [==============================] - 101s 2s/step - loss: 0.3272 - accuracy: 0.2090 - val_loss: 803.4706 - val_accuracy: 0.1702
Epoch 8